In [1]:
#import all the necessary libraries
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value
import pandas as pd
import numpy as np

Part 1: Two Stage DEA

In [2]:
#define a function to solve the DEA problem using PuLP
def calculate_efficiency(inputs, outputs):
    #Determine the deminsion of the problem by inspecting number of imports, outputs and DMUs
    num_dmu = inputs.shape[0]
    num_inputs = inputs.shape[1]
    num_outputs = outputs.shape[1]
    #Initiate the variables for storing the results
    efficiency_scores = np.zeros(num_dmu)

    for i in range(num_dmu):
        # create the LP problem
        model = LpProblem(f"DEA_DMU_{i}", LpMinimize)

        # define the variables
        theta = LpVariable("theta", lowBound=0)  # efficiency score
        lambdas = [LpVariable(f"lambda_{j}", lowBound=0) for j in range(num_dmu)] # efficiency weights

        # objective function: minimise the efficiency score
        model += theta

        # Constraint 1: Limit the total weighted sum of all DMU input to be less than θ times the inputs of the evaluated DMU
        for j in range(num_inputs):
            model += lpSum(lambdas[k] * inputs[k, j] for k in range(num_dmu)) <= theta * inputs[i, j]

        # Constraint 2: Ensure that the weighted sum of all DMU output is greater or equal to the evaluated DMU
        for j in range(num_outputs):
            model += lpSum(lambdas[k] * outputs[k, j] for k in range(num_dmu)) >= outputs[i, j]
        #solve the problem and store the efficiency score
        model.solve()
        efficiency_scores[i] = value(theta)

    return efficiency_scores

In [3]:
# read 2023 data
df_2023 = pd.read_excel("DATA.xlsx", sheet_name="2023")
# Invert the values of the Airline_CPE column
df_2023['Airline_CPE'] = 1/df_2023['Airline_CPE']

# Stage one inputs and outputs
inputs_stage1 = df_2023[['operating_expense', 'Terminal_Size']].values
outputs_stage1 = df_2023[['Total_Aircraft_Movements', 'Enplanement']].values
# Calculate stage one efficiency scores
df_2023['Airport_Efficiency'] = calculate_efficiency(inputs_stage1, outputs_stage1)

# Stage two inputs and outputs
inputs_stage2 = df_2023[['Airport_Efficiency','Operation_Cost']].values
outputs_stage2 = df_2023[['On_Time_Performance_Rate','Airline_CPE']].values

# Calculate stage two efficiency scores
df_2023['Airline_Efficiency'] = calculate_efficiency(inputs_stage2, outputs_stage2)
df_2023['Overall_Efficiency'] = df_2023['Airline_Efficiency'] * df_2023['Airport_Efficiency']

# Save the results as CSV
df_2023[['Hub Airport','Airline', 'Airport_Efficiency', 'Airline_Efficiency', 'Overall_Efficiency']].to_csv("dea_results_2023.csv", index=False)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/qq/qm3gfqv10wlf0kzp_9ql0hsc0000gn/T/a1c35ca7bc2c47c88214f3a56ab89a1c-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/qq/qm3gfqv10wlf0kzp_9ql0hsc0000gn/T/a1c35ca7bc2c47c88214f3a56ab89a1c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 133 RHS
At line 138 BOUNDS
At line 139 ENDATA
Problem MODEL has 4 rows, 31 columns and 122 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (0) rows, 31 (0) columns and 122 (0) elements
Perturbing problem by 0.001% of 6.0244293 - largest nonzero change 2.6861018e-05 ( 0.00044586825%) - largest zero change 1.5314754e-05
0  Obj 0 Primal inf 10.21783 (2)
6  Obj 0.69302432
Optimal - objective value 0.69299985
Optima

In [4]:
df_2023[['Hub Airport','Airline', 'Airport_Efficiency', 'Airline_Efficiency','Overall_Efficiency']]

,Hub Airport,Airline,Airport_Efficiency,Airline_Efficiency,Overall_Efficiency
0,Chicago Midway International Airport,SouthWest Airlines,0.693000,0.306330,0.212286
1,Hartsfield-Jackson Atlanta International Airport,SouthWest Airlines,0.592817,0.358098,0.212286
2,Denver International Airport,Frontier Airlines,0.774978,0.046146,0.035762
3,Dallas/Ft Worth International Airport,Frontier Airlines,0.350135,0.102137,0.035762
4,Minneapolis-Saint Paul International Airport,Sun Country Airlines,0.307444,0.080526,0.024757
5,Abudabi Airport,Air Arabia,0.386329,0.054348,0.020996
6,John F. Kennedy International Airport,JetBlue Airways,0.097501,0.722021,0.070398
7,Ankara Esenboga Airport,Anadolu Jet,0.209566,0.495480,0.103836
8,Ankara Esenboga Airport,Pegasus Airlines,0.217960,0.253008,0.055146
9,Dusseldorf Airport,Eurowings,0.162459,0.189268,0.030748


In [5]:
# read 2022 data
df_2022 = pd.read_excel("DATA.xlsx", sheet_name="2022")
df_2022['Airline_CPE'] = 1/df_2022['Airline_CPE']

# Stage 1 DEA calculation
inputs_stage1 = df_2022[['operating_expense', 'Terminal_Size']].values
outputs_stage1 = df_2022[['Total_Aircraft_Movements', 'Enplanement']].values

df_2022['Airport_Efficiency'] = calculate_efficiency(inputs_stage1, outputs_stage1)

# Stage 2 DEA calculation
inputs_stage2 = df_2022[['Airport_Efficiency','Operation_Cost']].values
outputs_stage2 = df_2022[['On_Time_Performance_Rate','Airline_CPE']].values

# Calculate stage two efficiency scores
df_2022['Airline_Efficiency'] = calculate_efficiency(inputs_stage2, outputs_stage2)
df_2022['Overall_Efficiency'] = df_2022['Airline_Efficiency'] * df_2022['Airport_Efficiency']

# Save the results as CSV
df_2022[['Hub Airport','Airline', 'Airport_Efficiency', 'Airline_Efficiency', 'Overall_Efficiency']].to_csv("dea_results_2022.csv", index=False)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/qq/qm3gfqv10wlf0kzp_9ql0hsc0000gn/T/cde95cd1c22a4fcca7b1571498e799a4-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/qq/qm3gfqv10wlf0kzp_9ql0hsc0000gn/T/cde95cd1c22a4fcca7b1571498e799a4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 133 RHS
At line 138 BOUNDS
At line 139 ENDATA
Problem MODEL has 4 rows, 31 columns and 122 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 4 (0) rows, 31 (0) columns and 122 (0) elements
Perturbing problem by 0.001% of 6.6581377 - largest nonzero change 2.9316055e-05 ( 0.00044030413%) - largest zero change 1.5920397e-05
0  Obj 0 Primal inf 18.790759 (2)
6  Obj 1.0000415
Optimal - objective value 1
Optimal objecti

In [6]:
df_2022[['Hub Airport','Airline', 'Airport_Efficiency', 'Airline_Efficiency','Overall_Efficiency']]

,Hub Airport,Airline,Airport_Efficiency,Airline_Efficiency,Overall_Efficiency
0,Chicago Midway International Airport,SouthWest Airlines,1.000000,0.148616,0.148616
1,Hartsfield-Jackson Atlanta International Airport,SouthWest Airlines,0.574333,0.258762,0.148616
2,Denver International Airport,Frontier Airlines,0.696795,0.013584,0.009465
3,Dallas/Ft Worth International Airport,Frontier Airlines,0.344595,0.027468,0.009465
4,Minneapolis-Saint Paul International Airport,Sun Country Airlines,0.296529,0.043078,0.012774
5,Abudabi Airport,Air Arabia,0.366179,0.026926,0.009860
6,John F. Kennedy International Airport,JetBlue Airways,0.070238,0.735790,0.051680
7,Ankara Esenboga Airport,Anadolu Jet,0.179593,0.299420,0.053774
8,Ankara Esenboga Airport,Pegasus Airlines,0.240897,0.174027,0.041923
9,Dusseldorf Airport,Eurowings,0.159516,0.136780,0.021819


In [7]:
# Define a function to calculate the Malmquist index
def malmquist_index(inputs_t1, outputs_t1, inputs_t2, outputs_t2):
    # Calculate efficiency scores inside each stage
    eff_t1 = calculate_efficiency(inputs_t1, outputs_t1)
    eff_t2 = calculate_efficiency(inputs_t2, outputs_t2)

    # Calculate efficiency scores between stages
    eff_t1_t2 = calculate_efficiency(inputs_t1, outputs_t2)
    eff_t2_t1 = calculate_efficiency(inputs_t2, outputs_t1)

    # Calculate efficiency change and technology change
    EC = eff_t2 / eff_t1
    TC = np.sqrt((eff_t1_t2 / eff_t2) * (eff_t1 / eff_t2_t1))
    # Calulate Malmquist index
    MI = EC * TC
    return MI, EC, TC

In [8]:
# Generate the inputs
inputs_2022_1 = df_2022[['operating_expense', 'Terminal_Size']]
inputs_2023_1 = df_2023[['operating_expense', 'Terminal_Size']]
outputs_2022_1 = df_2022[['Total_Aircraft_Movements','Total_Passenger_Traffic']]
outputs_2023_1 = df_2023[['Total_Aircraft_Movements','Total_Passenger_Traffic']]
inputs_2022_2 = df_2022[['Total_Aircraft_Movements','Total_Passenger_Traffic','Operation_Cost']]
inputs_2023_2 = df_2023[['Total_Aircraft_Movements','Total_Passenger_Traffic','Operation_Cost']]
outputs_2022_2 = df_2022[['Airline_CPE', 'On_Time_Performance_Rate']]
outputs_2023_2 = df_2023[['Airline_CPE', 'On_Time_Performance_Rate']]

# Convert the dataframes to numpy arrays
inputs_2022_1 = inputs_2022_1.to_numpy()
inputs_2023_1 = inputs_2023_1.to_numpy()
inputs_2022_2 = inputs_2022_2.to_numpy()
inputs_2023_2 = inputs_2023_2.to_numpy()
outputs_2022_1 = outputs_2022_1.to_numpy()
outputs_2023_1 = outputs_2023_1.to_numpy()
outputs_2022_2 = outputs_2022_2.to_numpy()
outputs_2023_2 = outputs_2023_2.to_numpy()


# Calculate the Malmquist index
MI_Stage1, EC_Stage1, TC_Stage1 = malmquist_index(inputs_2022_1, outputs_2022_1, inputs_2023_1, outputs_2023_1)

print("Malmquist Index (MI):", MI_Stage1)
print("Efficiency Change (EC):", EC_Stage1)
print("Technological Change (TC):", TC_Stage1)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/qq/qm3gfqv10wlf0kzp_9ql0hsc0000gn/T/c82cdd6e13644902b242b4fa2a94ca27-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/qq/qm3gfqv10wlf0kzp_9ql0hsc0000gn/T/c82cdd6e13644902b242b4fa2a94ca27-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 133 RHS
At line 138 BOUNDS
At line 139 ENDATA
Problem MODEL has 4 rows, 31 columns and 122 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (-1) rows, 24 (-7) columns and 71 (-51) elements
0  Obj 0 Primal inf 1.3049976 (2)
4  Obj 0.62608969
Optimal - objective value 0.62608969
After Postsolve, objective 0.62608969, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0.6260896853 - 4 iterations time 0.00

In [9]:
MI_Stage2, EC_Stage2, TC_Stage2 = malmquist_index(inputs_2022_2, outputs_2022_2, inputs_2023_2, outputs_2023_2)

print("Malmquist Index (MI):", MI_Stage2)
print("Efficiency Change (EC):", EC_Stage2)
print("Technological Change (TC):", TC_Stage2)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/qq/qm3gfqv10wlf0kzp_9ql0hsc0000gn/T/ca76010bc9b64be2a69c706522d3ae76-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/qq/qm3gfqv10wlf0kzp_9ql0hsc0000gn/T/ca76010bc9b64be2a69c706522d3ae76-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 165 RHS
At line 171 BOUNDS
At line 172 ENDATA
Problem MODEL has 5 rows, 31 columns and 153 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 5 (0) rows, 31 (0) columns and 153 (0) elements
Perturbing problem by 0.001% of 38.718421 - largest nonzero change 3.0704037e-05 ( 7.930085e-05%) - largest zero change 2.3280412e-05
0  Obj 0 Primal inf 1.0072867 (2)
7  Obj 1.0000026
Optimal - objective value 1
Optimal objecti

In [10]:
Final_MI = MI_Stage1 * MI_Stage2
Final_MI

array([ 0.93084102,  1.2914316 ,  2.96124375,  2.63096796,  0.99160147,
        1.67584459,  1.49921598,  1.35613913,  0.98212248,  0.74970269,
        0.67692616,  0.9353484 ,  0.75912004,  1.14519252,  1.30442732,
        1.10751611,  0.77909934,  0.65543099,  0.76598608,  1.85773261,
       13.08584765,  1.99151607,  2.33796124,  2.47163154,  0.86215891,
       13.5540653 ,  0.67760075,  0.87906667,  0.85792253,  1.43928625])